!pip install rrcf

In [1]:
import numpy as np
import pandas as pd

In [2]:
from utils.datasets import GhlKasperskyDataset, TepHarvardDataset, TepKasperskyDataset
from utils.custom_plots import plot_stacked

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

In [4]:
import rrcf

In [5]:
from adtk.metrics import precision, recall, f1_score
from utils.metrics import time_span_metrics

In [6]:
from tqdm.notebook import tqdm

# Get data from datasets

In [7]:
ds_ghl = GhlKasperskyDataset()
ds_ghl.shake_not_stir(valid_test_ratio=0.3)
train_ghl, _, _= next(ds_ghl.train_generator())
valid_ghl, faults_ghl, info_ghl= next(ds_ghl.valid_generator())

In [8]:
ds_tep1 = TepHarvardDataset()
ds_tep1.shake_not_stir(balanced_test=True, valid_test_ratio=1.0)
train_tep1, _, _= next(ds_tep1.train_generator())
valid_tep1, faults_tep1, info_tep1= next(ds_tep1.valid_generator())

In [9]:
ds_tep2 = TepHarvardDataset()
ds_tep2.shake_not_stir(valid_test_ratio=0.3)
train_tep2, _, _= next(ds_tep2.train_generator())
valid_tep2, faults_tep2, info_tep2= next(ds_tep2.valid_generator())

# IsolationForest test

## GHL

In [10]:
isoforest = IsolationForest(n_estimators=1000,
                            max_samples='auto',
                            contamination='auto',
                            max_features=1.0,
                            bootstrap=False,
                            n_jobs=-1,
                            random_state=31,
                            verbose=1,
                            warm_start=False)

In [11]:
isoforest.fit(train_ghl)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    3.0s remaining:    9.2s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    3.1s finished


IsolationForest(n_estimators=1000, n_jobs=-1, random_state=31, verbose=1)

In [12]:
detect = pd.Series(index=valid_ghl.index, data=isoforest.predict(valid_ghl))
detect.replace({1: 0, -1: 1}, inplace=True)

In [13]:
time_span_metrics(faults_ghl, detect)

(0.020100502512562814, 1.0, 0.03940886699507389)

In [14]:
# Generate data
n = 730
A = 50
center = 100
phi = 30
T = 2*np.pi/100
t = np.arange(n)
sin = A*np.sin(T*t-phi*T) + center
sin[235:255] = 80

# Set tree parameters
num_trees = 40
shingle_size = 4
tree_size = 256

# Create a forest of empty trees
forest = []
for _ in range(num_trees):
    tree = rrcf.RCTree()
    forest.append(tree)
    
# Use the "shingle" generator to create rolling window
points = rrcf.shingle(sin, size=shingle_size)

# Create a dict to store anomaly score of each point
avg_codisp = {}

# For each shingle...
for index, point in enumerate(points):
    # For each tree in the forest...
    for tree in forest:
        # If tree is above permitted size, drop the oldest point (FIFO)
        if len(tree.leaves) > tree_size:
            tree.forget_point(index - tree_size)
        # Insert the new point into the tree
        tree.insert_point(point, index=index)
        # Compute codisp on the new point and take the average among all trees
        if not index in avg_codisp:
            avg_codisp[index] = 0
        avg_codisp[index] += tree.codisp(index) / num_trees

## TEP Harvard

In [15]:
isoforest = IsolationForest(n_estimators=100,
                            max_samples='auto',
                            contamination='auto',
                            max_features=1.0,
                            bootstrap=False,
                            n_jobs=-1,
                            random_state=31,
                            verbose=1,
                            warm_start=False)

In [16]:
isoforest.fit(train_tep1.values)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.1s remaining:    0.6s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished


IsolationForest(n_jobs=-1, random_state=31, verbose=1)

In [17]:
detect = pd.Series(index=valid_tep1.index, data=isoforest.predict(valid_tep1.values))
detect.replace({1: 0, -1: 1}, inplace=True)

In [18]:
time_span_metrics(faults_tep1, detect)

(1.0, 1.0, 1.0)

## TEP Kaspersky

In [19]:
isoforest = IsolationForest(n_estimators=100,
                            max_samples='auto',
                            contamination='auto',
                            max_features=1.0,
                            bootstrap=False,
                            n_jobs=-1,
                            random_state=31,
                            verbose=1,
                            warm_start=False)

In [20]:
isoforest.fit(train_tep2.values)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.2s remaining:    0.7s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.2s finished


IsolationForest(n_jobs=-1, random_state=31, verbose=1)

In [21]:
detect = pd.Series(index=valid_tep2.index, data=isoforest.predict(valid_tep2.values))
detect.replace({1: 0, -1: 1}, inplace=True)

In [22]:
time_span_metrics(faults_tep2, detect)

(1.0, 0.7916666666666666, 0.8837209302325582)

# Isolating Watchman test

In [23]:
from utils.watchmen import IsolatingWatchman

## GHL

In [24]:
watchman_ghl = IsolatingWatchman(random_state=31)

In [25]:
ds_ghl.shake_not_stir(valid_test_ratio=0.3)

In [26]:
# learn data
train_gen = ds_ghl.train_generator()
for train, _, _ in tqdm(train_gen):
    watchman_ghl.partial_fit(train)

0it [00:00, ?it/s]

In [27]:
train.shape

(25001, 12)

In [28]:
# examine
valid_gen = ds_ghl.valid_generator()
examine_list = pd.DataFrame(columns=['precision', 'recall', 'f1_score'], dtype='float')
for valid, faults, info in tqdm(valid_gen):
    detect = watchman_ghl.predict(valid)
    examine_list.loc[info] = time_span_metrics(faults, detect)
print(examine_list.mean())

0it [00:00, ?it/s]

precision    0.014502
recall       0.811054
f1_score     0.028326
dtype: float64


In [29]:
watchman_ghl

IsolatingWatchman(n_trees=1000)

## TEP Harvard

In [30]:
watchman_tep1 = IsolatingWatchman(random_state=31)

In [31]:
ds_tep1.shake_not_stir(balanced_test=True, valid_test_ratio=0.5)

In [32]:
# learn data
train_gen = ds_tep1.train_generator()
for train, _, _ in tqdm(train_gen, total=500):
    watchman_tep1.partial_fit(train)

  0%|          | 0/500 [00:00<?, ?it/s]

In [33]:
# examine
valid_gen = ds_tep1.valid_generator()
examine_list = pd.DataFrame(columns=['precision', 'recall', 'f1_score'], dtype='float')
for valid, faults, info in tqdm(valid_gen, total=500):
    detect = watchman_tep1.predict(valid)
    examine_list.loc[info] = time_span_metrics(faults, detect)
#     if max(faults) and max(detect_vector) and 'fault_011' in info:
#         plot_stacked(test, faults=faults, detect=detect_vector)
#         break
print(examine_list.mean())

  0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
watchman_tep1

# RRCF test

In [153]:
import rrcf

In [154]:
# from utils.watchmen import CuttingWatchman

from typing import Optional

class CuttingWatchman:
    # using Robust Random Cut Forest for anomaly detection

    def __init__(self,
                 max_trees: int = 1000,
                 tree_size: int = 256,
                 random_state: Optional[int] = None
                 ):
        self.forest = []
        self.max_trees = max_trees
        self.tree_size = tree_size
        self.max_codisp = 0.0
        if isinstance(random_state, int):
            np.random.seed(random_state)
        return

    def __repr__(self):
        return f'{self.__class__.__name__}(n_trees={len(self.forest)})'

    def partial_fit(self, data: pd.DataFrame, tolerance: float = 0.05) -> None:
        # choose how many trees need to add
        for _ in range(data.shape[0] // self.tree_size):
            if len(self.forest) < self.max_trees:
                # add new tree with tree_size
                sample = data.sample(n=self.tree_size)
                try:
                    tree = rrcf.RCTree(X=sample.values)
                except:
                    # bug in framework with some samples
                    # try again
                    sample = data.sample(n=self.tree_size)
                    tree = rrcf.RCTree(X=sample.values)
                self.forest.append(tree)
                # compute codisp for all leaves in new tree and store max
                max_codisp = max([tree.codisp(l) for l in tree.leaves])
                self.max_codisp = max(self.max_codisp, max_codisp * (1 + tolerance))
        return

    def predict(self, data: pd.DataFrame) -> pd.DataFrame:
        codisp = pd.Series(index=range(data.shape[0]), data=0.0)  # average codisp over forest
        # add new points to copies of trees and compute codisp
        for tree in self.forest:
            # test_tree = rrcf.RCTree.from_dict(tree.to_dict())
            for i in range(data.shape[0]):
                tree.insert_point(data.iloc[i], index=i+self.tree_size)
                codisp[i] += test_tree.codisp(i+self.tree_size)
                tree.forget_point(index=i+self.tree_size)
        codisp /= self.tree_size
        codisp.index = data.index
        result = (codisp > self.max_codisp).astype('unit8')
        return result


## GHL

In [172]:
watchman_ghl = CuttingWatchman(random_state=31)

In [173]:
ds_ghl.shake_not_stir(valid_test_ratio=0.3)

In [174]:
# learn data
train_gen = ds_ghl.train_generator()
for train, _, _ in tqdm(train_gen):
    watchman_ghl.partial_fit(train, tolerance=0.05)
watchman_ghl.max_codisp

0it [00:00, ?it/s]

267.75

In [175]:
watchman_ghl

CuttingWatchman(n_trees=97)

In [176]:
# examine
valid_gen = ds_ghl.valid_generator()
examine_list = pd.DataFrame(columns=['precision', 'recall', 'f1_score'], dtype='float')
for valid, faults, info in tqdm(valid_gen):
    detect = watchman_ghl.predict(valid)
    examine_list.loc[info] = time_span_metrics(faults, detect)
print(examine_list.mean())

0it [00:00, ?it/s]

KeyError: 'leaf must be a Leaf instance or key to self.leaves'

In [180]:
valid.shape[0] * len(watchman_ghl.forest)

323398

In [182]:
codisp = pd.Series(index=range(valid.shape[0]), data=0.0)
for tree in watchman_ghl.forest:
    test_tree = rrcf.RCTree.from_dict(tree.to_dict())
#     for i in range(valid.shape[0]):
#         j = max(tree.leaves) + 1 + i
#         tree.insert_point(valid.iloc[i], index=j)
#         codisp[i] += tree.codisp(j)
#         tree.forget_point(index=j)

# it takes toooooo long

In [170]:
tree.leaves[257]

Leaf(256)